In [1]:
import cv2
import numpy as np
import pandas as pd
%matplotlib inline

In [2]:
def disp(title, data) : 
    cv2.imshow(title, data)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [3]:
data = cv2.imread('./Test/Before.png')
disp("Jamie_Before", data)

In [4]:
eye_c = cv2.CascadeClassifier("./Train/third-party/frontalEyes35x16.xml")
nose_c = cv2.CascadeClassifier("./Train/third-party/Nose18x15.xml")

if eye_c.empty():
    raise IOError('Unable to load the eye cascade classifier xml file')
    
if nose_c.empty():
    raise IOError('Unable to load the nose cascade classifier xml file')

In [5]:
eyes = eye_c.detectMultiScale(data,1.3,5)
nose = nose_c.detectMultiScale(data,1.3,5)

xe, ye, we, he = eyes[0]
xn, yn, wn, hn = nose[0]

cv2.rectangle(data, (xe,ye), (xe+we,ye+he), (0,255,0), 3)
    
disp("I", data)

In [6]:
glasses = cv2.imread("./Train/glasses.png",cv2.IMREAD_UNCHANGED)
mustache = cv2.imread("./Train/mustache.png",cv2.IMREAD_UNCHANGED)

glasses = cv2.resize(glasses, (we, he))
mustache = cv2.resize(mustache, (wn, hn))
disp("Glasses", glasses)
disp("Mustache", mustache)

print(glasses.shape)

(59, 130, 4)


In [7]:
output = data.copy()

In [8]:
alpha_glass = glasses[:, :, 3] / 255.0
alpha_mustache = mustache[:, :, 3] / 255.0

alpha_g_image = 1.0 - alpha_glass
alpha_m_image = 1.0 - alpha_mustache

x_e_offset = xe
y_e_offset = ye
ye1, ye2 = y_e_offset, y_e_offset + glasses.shape[0]
xe1, xe2 = x_e_offset, x_e_offset + glasses.shape[1]

x_n_offset = xn
y_n_offset = yn
yn1, yn2 = y_n_offset, y_n_offset + mustache.shape[0]
xn1, xn2 = x_n_offset, x_n_offset + mustache.shape[1]


for c in range(0, 3):
    output[ye1:ye2, xe1:xe2, c] = ((alpha_g_image*output[ye1:ye2, xe1:xe2, c]) +
                                   (alpha_glass*glasses[:, :, c]))

for c in range(0, 3):
    output[yn1:yn2, xn1:xn2, c] = ((alpha_m_image*output[yn1:yn2, xn1:xn2, c]) +
                                   (alpha_mustache*mustache[:, :, c]))

In [9]:
print(output.shape)
disp("image", output)
cv2.imwrite("outputimage.jpeg", output)

output_np = np.array(output)
output_b = output_np[:, :, 0]
output_g = output_np[:, :, 1]
output_r = output_np[:, :, 2]

output_b = output_b.flatten()
output_g = output_g.flatten()
output_r = output_r.flatten()

out = np.stack([output_b, output_g, output_r])
out = out.reshape((-1, 3))

out_df = pd.DataFrame(out, columns=['Channel 1', 'Channel 2', 'Channel 3'])
out_df.to_csv('outputfile.csv', index=False, index_label=False)

(485, 377, 3)
